In [1]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    plt.close(fig)


def nearest_nonzero_idx(a,x,y):
    r,c = np.nonzero(a)
    min_idx = ((r - x)**2 + (c - y)**2).argmin()
    return r[min_idx], c[min_idx]


def emergence_pixels(gf, vel_x_raw, vel_y_raw, icethickness_raw, xres, yres, 
                     vel_min=0, max_velocity=600, vel_depth_avg_factor=0.8, option_border=1,
                     positive_is_east=True, positive_is_north=True, constant_icethickness=False, debug=True):
    """ Compute the emergence velocity using an ice flux approach
    """
    # Glacier mask
    glac_mask = np.zeros(vel_x_raw.shape) + 1
    glac_mask[gf.z1.mask] = 0
    
    # Modify vel_y by multiplying velocity by -1 such that matrix operations agree with flow direction
    #    Specifically, a negative y velocity means the pixel is flowing south.
    #    However, if you were to subtract that value from the rows, it would head north in the matrix.
    #    This is due to the fact that the number of rows start at 0 at the top.
    #    Therefore, multipylying by -1 aligns the matrix operations with the flow direction
    if positive_is_north:
        vel_y = -1*vel_y_raw * vel_depth_avg_factor
    else:
        vel_y = vel_y_raw * vel_depth_avg_factor
    if positive_is_east:
        vel_x = vel_x_raw * vel_depth_avg_factor
    else:
        vel_x = -1*vel_x_raw * vel_depth_avg_factor
    vel_total = (vel_y**2 + vel_x**2)**0.5
    # Ice thickness
    icethickness = icethickness_raw.copy()
    if constant_icethickness:
        icethickness[:,:] = 1
        icethickness = icethickness * glac_mask
#     print('mean ice thickness:', np.round(icethickness.mean(),0), 'm')
    # Compute the initial volume
    volume_initial = icethickness * (xres * yres)
    pix_maxres = xres
    if yres > pix_maxres:
        pix_maxres = yres
    # Quality control options:
    # Apply a border based on the max specified velocity to prevent errors associated with pixels going out of bounds
    if option_border == 1:
        border = int(max_velocity / pix_maxres) + 1
        for r in range(vel_x.shape[0]):
            for c in range(vel_x.shape[1]):
                if (r < border) | (r >= vel_x.shape[0] - border) | (c < border) | (c >= vel_x.shape[1] - border):
                    vel_x[r,c] = 0
                    vel_y[r,c] = 0
    # Minimum/maximum velocity bounds
    vel_x[vel_total < vel_min] = 0
    vel_y[vel_total < vel_min] = 0
    vel_x[vel_total > max_velocity] = 0
    vel_y[vel_total > max_velocity] = 0
#     # Remove clusters of high velocity on stagnant portions of glaciers due to feature tracking of ice cliffs and ponds
#     if option_stagnantbands == 1:
#         vel_x[bands <= stagnant_band] = 0
#         vel_y[bands <= stagnant_band] = 0        
    # Compute displacement in units of pixels
    vel_x_pix = vel_x / xres
    vel_y_pix = vel_y / yres
    # Compute the displacement and fraction of pixels moved for all columns (x-axis)
    # col_x1 is the number of columns to the closest pixel receiving ice [ex. 2.6 returns 2, -2.6 returns -2]
    #    int() automatically rounds towards zero
    col_x1 = vel_x_pix.astype(int)
    # col_x2 is the number of columns to the further pixel receiving ice [ex. 2.6 returns 3, -2.6 returns -3]
    #    np.sign() returns a value of 1 or -1, so it's adding 1 pixel away from zero
    col_x2 = (vel_x_pix + np.sign(vel_x_pix)).astype(int)
    # rem_x2 is the fraction of the pixel that remains in the further pixel (col_x2) [ex. 2.6 returns 0.6, -2.6 returns 0.6]
    #    np.sign() returns a value of 1 or -1, so multiplying by that ensures you have a positive value
    #    then when you take the remainder using "% 1", you obtain the desired fraction
    rem_x2 = np.multiply(np.sign(vel_x_pix), vel_x_pix) % 1
    # rem_x1 is the fraction of the pixel that remains in the closer pixel (col_x1) [ex. 2.6 returns 0.4, -2.6 returns 0.4]
    rem_x1 = 1 - rem_x2
    # Repeat the displacement and fraction computations for all rows (y-axis)
    row_y1 = vel_y_pix.astype(int)
    row_y2 = (vel_y_pix + np.sign(vel_y_pix)).astype(int)
    rem_y2 = np.multiply(np.sign(vel_y_pix), vel_y_pix) % 1
    rem_y1 = 1 - rem_y2
          
    # Compute the mass flux for each pixel
    volume_final = np.zeros(volume_initial.shape)
    for r in range(vel_x.shape[0]):
        for c in range(vel_x.shape[1]):
            volume_final[r+row_y1[r,c], c+col_x1[r,c]] = (
                volume_final[r+row_y1[r,c], c+col_x1[r,c]] + rem_y1[r,c]*rem_x1[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y2[r,c], c+col_x1[r,c]] = (
                volume_final[r+row_y2[r,c], c+col_x1[r,c]] + rem_y2[r,c]*rem_x1[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y1[r,c], c+col_x2[r,c]] = (
                volume_final[r+row_y1[r,c], c+col_x2[r,c]] + rem_y1[r,c]*rem_x2[r,c]*volume_initial[r,c]
                )
            volume_final[r+row_y2[r,c], c+col_x2[r,c]] = (
                volume_final[r+row_y2[r,c], c+col_x2[r,c]] + rem_y2[r,c]*rem_x2[r,c]*volume_initial[r,c]
                )
         
    # Redistribute off-glacier volume back onto the nearest pixel on the glacier
    offglac_row, offglac_col = np.where((glac_mask == 0) & (volume_final > 0))
    for nidx in range(0,len(offglac_row)):
        nrow = offglac_row[nidx]
        ncol = offglac_col[nidx]
        ridx, cidx = nearest_nonzero_idx(glac_mask, nrow, ncol)
        # Add off-glacier volume back onto nearest pixel on glacier
        volume_final[ridx,cidx] += volume_final[nrow,ncol]
        volume_final[nrow,ncol] = 0
            
    # Check that mass is conserved (threshold = 0.1 m x pixel_size**2)
    if debug:
        print('Mass is conserved?', np.absolute(volume_final.sum() - volume_initial.sum()) / volume_initial.sum() < 0.01)
        print(np.round(np.absolute(volume_final.sum() - volume_initial.sum()),1), 
              np.round(np.absolute(volume_final.sum() - volume_initial.sum()) / volume_initial.sum() * 100,2), '%')
        
    if np.absolute(volume_final.sum() - volume_initial.sum()) / volume_initial.sum() > 0.01:
        print('MASS NOT CONSERVED FOR EMERGENCE VELOCITY')
    # Final ice thickness
    icethickness_final = volume_final / (xres * yres)
    # Emergence velocity
    emergence_velocity = icethickness_final - icethickness
    return emergence_velocity



class GlacFeat:
    def __init__(self, feat, glacname_fieldname, glacnum_fieldname):

        self.glacname = feat.GetField(glacname_fieldname)
        if self.glacname is None:
            self.glacname = ""
        else:
            #RGI has some nonstandard characters
            #self.glacname = self.glacname.decode('unicode_escape').encode('ascii','ignore')
            #glacname = re.sub(r'[^\x00-\x7f]',r'', glacname)
            self.glacname = re.sub(r'\W+', '', self.glacname)
            self.glacname = self.glacname.replace(" ", "")
            self.glacname = self.glacname.replace("_", "")
            self.glacname = self.glacname.replace("/", "")

        self.glacnum = feat.GetField(glacnum_fieldname)
        fn = feat.GetDefnRef().GetName()
        #RGIId (String) = RGI50-01.00004
        self.glacnum = '%0.5f' % float(self.glacnum.split('-')[-1])

        if self.glacname:
            self.feat_fn = "%s_%s" % (self.glacnum, self.glacname)
        else:
            self.feat_fn = str(self.glacnum)

        self.glac_geom_orig = geolib.geom_dup(feat.GetGeometryRef())
        self.glac_geom = geolib.geom_dup(self.glac_geom_orig)
        #Hack to deal with fact that this is not preserved in geom when loaded from pickle on disk
        self.glac_geom_srs_wkt = self.glac_geom.GetSpatialReference().ExportToWkt()

        #Attributes written by mb_calc
        self.z1 = None
        self.z1_hs = None
        self.z1_stats = None
        self.z1_ela = None
        self.z2 = None
        self.z2_hs = None
        self.z2_stats = None
        self.z2_ela = None
        self.z2_aspect = None
        self.z2_aspect_stats = None
        self.z2_slope = None
        self.z2_slope_stats = None
        self.res = None
        self.dhdt = None
        self.mb = None
        self.mb_mean = None
        self.t1 = None
        self.t2 = None
        self.dt = None
        self.t1_mean = None
        self.t2_mean = None
        self.dt_mean = None

        self.H = None
        self.H_mean = np.nan
        self.vx = None
        self.vy = None
        self.vm = None
        self.vm_mean = np.nan
        self.divQ = None
        self.emvel = None
        self.debris_class = None
        self.debris_thick = None
        self.debris_thick_mean = np.nan
        self.perc_clean = np.nan
        self.perc_debris = np.nan
        self.perc_pond = np.nan

    def geom_srs_update(self, srs=None):
        if self.glac_geom.GetSpatialReference() is None:
            if srs is None:
                srs = osr.SpatialReference()
                srs.ImportFromWkt(self.glac_geom_srs_wkt)
            self.glac_geom.AssignSpatialReference(srs)

    def geom_attributes(self, srs=None):
        self.geom_srs_update()
        if srs is not None:
            #Should reproject here to equal area, before geom_attributes
            #self.glac_geom.AssignSpatialReference(glac_shp_srs)
            #self.glac_geom_local = geolib.geom2localortho(self.glac_geom)
            geolib.geom_transform(self.glac_geom, srs)

        self.glac_geom_extent = geolib.geom_extent(self.glac_geom)
        self.glac_area = self.glac_geom.GetArea()
        self.glac_area_km2 = self.glac_area / 1E6
        self.cx, self.cy = self.glac_geom.Centroid().GetPoint_2D()
        
        
#RGI uses 50 m bins
def hist_plot(gf, bin_width=50.0, dz_clim=(-2.0, 2.0), exportcsv=True, csv_ending=''):
    #print("Generating histograms")
    #Create bins for full range of input data and specified bin width

    #NOTE: these counts/areas are for valid pixels only
    #Not necessarily a true representation of actual glacier hypsometry
    #Need a void-filled DEM for this

    z_bin_edges, z_bin_centers = malib.get_bins(gf.z1, bin_width)
    #Need to compress here, otherwise histogram uses masked values!
    z1_bin_counts, z1_bin_edges = np.histogram(gf.z1.compressed(), bins=z_bin_edges)
    z1_bin_areas = z1_bin_counts * gf.res[0] * gf.res[1] / 1E6
    #RGI standard is integer thousandths of glaciers total area
    #Should check to make sure sum of bin areas equals total area
    #z1_bin_areas_perc = 100. * z1_bin_areas / np.sum(z1_bin_areas)
    z1_bin_areas_perc = 100. * (z1_bin_areas / gf.glac_area_km2)

    #If we only have one elevation grid with dhdt
    if gf.z2 is not None:
        z2_bin_counts, z2_bin_edges = np.histogram(gf.z2.compressed(), bins=z_bin_edges)
        z2_bin_areas = z2_bin_counts * gf.res[0] * gf.res[1] / 1E6
        #z2_bin_areas_perc = 100. * z2_bin_areas / np.sum(z2_bin_areas)
        z2_bin_areas_perc = 100. * (z1_bin_areas / gf.glac_area_km2)
    else:
        z2_bin_counts = z1_bin_counts
        z2_bin_edges = z1_bin_edges
        z2_bin_areas = z1_bin_areas
        z2_bin_areas_perc = z1_bin_areas_perc

    #Create arrays to store output
    slope_bin_med = np.ma.masked_all_like(z1_bin_areas)
    slope_bin_mad = np.ma.masked_all_like(z1_bin_areas)
    aspect_bin_med = np.ma.masked_all_like(z1_bin_areas)
    aspect_bin_mad = np.ma.masked_all_like(z1_bin_areas)
    if gf.dhdt is not None:
        mb_bin_med = np.ma.masked_all_like(z1_bin_areas)
        np.ma.set_fill_value(mb_bin_med, np.nan)
        mb_bin_mad = np.ma.masked_all_like(mb_bin_med)
        mb_bin_mean = np.ma.masked_all_like(mb_bin_med)
        mb_bin_std = np.ma.masked_all_like(mb_bin_med)
        dhdt_bin_med = np.ma.masked_all_like(mb_bin_med)
        dhdt_bin_mad = np.ma.masked_all_like(mb_bin_med)
        dhdt_bin_mean = np.ma.masked_all_like(mb_bin_med)
        dhdt_bin_std = np.ma.masked_all_like(mb_bin_med)
        dhdt_bin_count = np.ma.masked_all_like(mb_bin_med)
    if gf.vm is not None:
        vm_bin_med = np.ma.masked_all_like(z1_bin_areas)
        vm_bin_mad = np.ma.masked_all_like(z1_bin_areas)
    if gf.H is not None:
        H_bin_mean = np.ma.masked_all_like(z1_bin_areas)
        H_bin_std = np.ma.masked_all_like(z1_bin_areas)
    if gf.emvel is not None:
        emvel_bin_mean = np.ma.masked_all_like(z1_bin_areas)
        emvel_bin_std = np.ma.masked_all_like(z1_bin_areas)
        emvel_bin_med = np.ma.masked_all_like(z1_bin_areas)
        emvel_bin_mad = np.ma.masked_all_like(z1_bin_areas)
    if gf.debris_class is not None:
#         perc_clean = np.ma.masked_all_like(z1_bin_areas)
#         perc_debris = np.ma.masked_all_like(z1_bin_areas)
#         perc_pond = np.ma.masked_all_like(z1_bin_areas)
        debris_thick_med = np.ma.masked_all_like(z1_bin_areas)
        debris_thick_mad = np.ma.masked_all_like(z1_bin_areas)
#         dhdt_clean_bin_med = np.ma.masked_all_like(z1_bin_areas)
#         dhdt_debris_bin_med = np.ma.masked_all_like(z1_bin_areas)
#         dhdt_pond_bin_med = np.ma.masked_all_like(mz1_bin_areas)

#         gf.dhdt_clean = np.ma.array(gf.dhdt, mask=~((gf.debris_class == 1).data))
#         gf.dhdt_debris = np.ma.array(gf.dhdt, mask=~((gf.debris_class == 2).data))
#         gf.dhdt_pond = np.ma.array(gf.dhdt, mask=~((gf.debris_class == 3).data))

    if gf.debris_thick_ts is not None:
        debris_thick_ts_med = np.ma.masked_all_like(z1_bin_areas)
        debris_thick_ts_mad = np.ma.masked_all_like(z1_bin_areas)
    if gf.meltfactor_ts is not None:
        meltfactor_ts_med = np.ma.masked_all_like(z1_bin_areas)
        meltfactor_ts_mad = np.ma.masked_all_like(z1_bin_areas)

    #Bin sample count must be greater than this value
    min_bin_samp_count = 9

    #Loop through each bin and extract stats
    idx = np.digitize(gf.z1, z_bin_edges)
    for bin_n in range(z_bin_centers.size):
        if gf.dhdt is not None:
            mb_bin_samp = gf.mb_map[(idx == bin_n+1)]
            if mb_bin_samp.count() > min_bin_samp_count:
                mb_bin_med[bin_n] = malib.fast_median(mb_bin_samp)
                mb_bin_mad[bin_n] = malib.mad(mb_bin_samp)
                mb_bin_mean[bin_n] = mb_bin_samp.mean()
                mb_bin_std[bin_n] = mb_bin_samp.std()
            dhdt_bin_samp = gf.dhdt[(idx == bin_n+1)]
            if dhdt_bin_samp.count() > min_bin_samp_count:
                dhdt_bin_med[bin_n] = malib.fast_median(dhdt_bin_samp)
                dhdt_bin_mad[bin_n] = malib.mad(dhdt_bin_samp)
                dhdt_bin_mean[bin_n] = dhdt_bin_samp.mean()
                dhdt_bin_std[bin_n] = dhdt_bin_samp.std()
                dhdt_bin_count[bin_n] = dhdt_bin_samp.count()
        if gf.debris_thick is not None:
            debris_thick_bin_samp = gf.debris_thick[(idx == bin_n+1)]
            if debris_thick_bin_samp.size > min_bin_samp_count:
                debris_thick_med[bin_n] = malib.fast_median(debris_thick_bin_samp)
                debris_thick_mad[bin_n] = malib.mad(debris_thick_bin_samp)
        
        if gf.debris_thick_ts is not None:
            debris_thick_ts_bin_samp = gf.debris_thick_ts[(idx == bin_n+1)]
            if debris_thick_ts_bin_samp.size > min_bin_samp_count:
                debris_thick_ts_med[bin_n] = malib.fast_median(debris_thick_ts_bin_samp)
                debris_thick_ts_mad[bin_n] = malib.mad(debris_thick_ts_bin_samp)
        if gf.meltfactor_ts is not None:
            meltfactor_ts_bin_samp = gf.meltfactor_ts[(idx == bin_n+1)]
            if meltfactor_ts_bin_samp.size > min_bin_samp_count:
                meltfactor_ts_med[bin_n] = malib.fast_median(meltfactor_ts_bin_samp)
                meltfactor_ts_mad[bin_n] = malib.mad(meltfactor_ts_bin_samp)
                
        if gf.debris_class is not None:
            debris_class_bin_samp = gf.debris_class[(idx == bin_n+1)]
            dhdt_clean_bin_samp = gf.dhdt_clean[(idx == bin_n+1)]
            dhdt_debris_bin_samp = gf.dhdt_debris[(idx == bin_n+1)]
            dhdt_pond_bin_samp = gf.dhdt_pond[(idx == bin_n+1)]
            if debris_class_bin_samp.count() > min_bin_samp_count:
                perc_clean[bin_n] = 100. * (debris_class_bin_samp == 1).sum()/debris_class_bin_samp.count()
                perc_debris[bin_n] = 100. * (debris_class_bin_samp == 2).sum()/debris_class_bin_samp.count()
                perc_pond[bin_n] = 100. * (debris_class_bin_samp == 3).sum()/debris_class_bin_samp.count()
            if dhdt_clean_bin_samp.count() > min_bin_samp_count:
                dhdt_clean_bin_med[bin_n] = malib.fast_median(dhdt_clean_bin_samp)
            if dhdt_debris_bin_samp.count() > min_bin_samp_count:
                dhdt_debris_bin_med[bin_n] = malib.fast_median(dhdt_debris_bin_samp)
            if dhdt_pond_bin_samp.count() > min_bin_samp_count:
                dhdt_pond_bin_med[bin_n] = malib.fast_median(dhdt_pond_bin_samp)
        if gf.vm is not None:
            vm_bin_samp = gf.vm[(idx == bin_n+1)]
            if vm_bin_samp.size > min_bin_samp_count:
                vm_bin_med[bin_n] = malib.fast_median(vm_bin_samp)
                vm_bin_mad[bin_n] = malib.mad(vm_bin_samp)
        if gf.H is not None:
            H_bin_samp = gf.H[(idx == bin_n+1)]
            if H_bin_samp.size > min_bin_samp_count:
                H_bin_mean[bin_n] = H_bin_samp.mean()
                H_bin_std[bin_n] = H_bin_samp.std()
        if gf.emvel is not None:
            emvel_bin_samp = gf.emvel[(idx == bin_n+1)]
            if emvel_bin_samp.size > min_bin_samp_count:
                emvel_bin_mean[bin_n] = emvel_bin_samp.mean()
                emvel_bin_std[bin_n] = emvel_bin_samp.std()
                emvel_bin_med[bin_n] = malib.fast_median(emvel_bin_samp)
                emvel_bin_mad[bin_n] = malib.mad(emvel_bin_samp)
        slope_bin_samp = gf.z1_slope[(idx == bin_n+1)]
        if slope_bin_samp.size > min_bin_samp_count:
            slope_bin_med[bin_n] = malib.fast_median(slope_bin_samp)
            slope_bin_mad[bin_n] = malib.mad(slope_bin_samp)
        aspect_bin_samp = gf.z1_aspect[(idx == bin_n+1)]
        if aspect_bin_samp.size > min_bin_samp_count:
            aspect_bin_med[bin_n] = malib.fast_median(aspect_bin_samp)
            aspect_bin_mad[bin_n] = malib.mad(aspect_bin_samp)

    if gf.dhdt is not None:
        dhdt_bin_areas = dhdt_bin_count * gf.res[0] * gf.res[1] / 1E6
        #dhdt_bin_areas_perc = 100. * dhdt_bin_areas / np.sum(dhdt_bin_areas)
        dhdt_bin_areas_perc = 100. * (dhdt_bin_areas / gf.glac_area_km2)

    outbins_header = 'bin_center_elev_m, z1_bin_count_valid, z1_bin_area_valid_km2, z1_bin_area_perc, z2_bin_count_valid, z2_bin_area_valid_km2, z2_bin_area_perc, slope_bin_med, aspect_bin_med'
    fmt = '%0.1f, %0.0f, %0.3f, %0.2f, %0.0f, %0.3f, %0.2f, %0.2f, %0.2f'
    outbins = [z_bin_centers, z1_bin_counts, z1_bin_areas, z1_bin_areas_perc, z2_bin_counts, z2_bin_areas, z2_bin_areas_perc, slope_bin_med, aspect_bin_med]
    if gf.dhdt is not None:
        outbins_header += ', dhdt_bin_count, dhdt_bin_area_valid_km2, dhdt_bin_area_perc, dhdt_bin_med_ma, dhdt_bin_mad_ma, dhdt_bin_mean_ma, dhdt_bin_std_ma, mb_bin_med_mwea, mb_bin_mad_mwea, mb_bin_mean_mwea, mb_bin_std_mwea'
        fmt += ', %0.0f, %0.3f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f'
        outbins.extend([dhdt_bin_count, dhdt_bin_areas, dhdt_bin_areas_perc, dhdt_bin_med, dhdt_bin_mad, dhdt_bin_mean, dhdt_bin_std, \
                        mb_bin_med, mb_bin_mad, mb_bin_mean, mb_bin_std])
    if gf.debris_thick is not None:
        outbins_header += ', debris_thick_med_m, debris_thick_mad_m'
        fmt += ', %0.2f, %0.2f'
        debris_thick_med[debris_thick_med == -(np.inf)] = 0.00
        debris_thick_mad[debris_thick_mad == -(np.inf)] = 0.00
        outbins.extend([debris_thick_med, debris_thick_mad])
    
    if gf.debris_thick_ts is not None:
        outbins_header += ',debris_thick_ts_med_m,debris_thick_ts_mad_m'
        fmt += ', %0.2f, %0.2f'
        debris_thick_ts_med[debris_thick_ts_med == -(np.inf)] = 0.00
        debris_thick_ts_mad[debris_thick_ts_mad == -(np.inf)] = 0.00
        outbins.extend([debris_thick_ts_med, debris_thick_ts_mad])
    if gf.meltfactor_ts is not None:
        outbins_header += ',meltfactor_ts_med_m,meltfactor_ts_mad_m'
        fmt += ', %0.2f, %0.2f'
        meltfactor_ts_med[meltfactor_ts_med == -(np.inf)] = 1
        meltfactor_ts_med[meltfactor_ts_med > 1] = 1
        meltfactor_ts_med[meltfactor_ts_med <= 0] = 1
        meltfactor_ts_mad[meltfactor_ts_mad == -(np.inf)] = 0
        meltfactor_ts_mad[meltfactor_ts_mad > 1] = 0
        meltfactor_ts_mad[meltfactor_ts_mad <= 0] = 0
        outbins.extend([meltfactor_ts_med, meltfactor_ts_mad])
    
    if gf.debris_class is not None:
        outbins_header += ', perc_debris, perc_pond, perc_clean, dhdt_debris_med, dhdt_pond_med, dhdt_clean_med'
        fmt += ', %0.2f, %0.2f, %0.2f, %0.2f, %0.2f, %0.2f'
        outbins.extend([perc_debris, perc_pond, perc_clean, dhdt_debris_bin_med, dhdt_pond_bin_med, dhdt_clean_bin_med])
    if gf.vm is not None:
        outbins_header += ', vm_med, vm_mad'
        fmt += ', %0.2f, %0.2f'
        outbins.extend([vm_bin_med, vm_bin_mad])
    if gf.H is not None:
        outbins_header += ', H_mean, H_std'
        fmt += ', %0.2f, %0.2f'
        outbins.extend([H_bin_mean, H_bin_std])
#         outbins_header += ', H_mean, H_std, emvel_mean, emvel_std'
#         fmt += ', %0.2f, %0.2f, %0.2f, %0.2f'
#         outbins.extend([H_bin_mean, H_bin_std, emvel_bin_mean, emvel_bin_std])

    if gf.emvel is not None:
        outbins_header += ', emvel_mean, emvel_std, emvel_med, emvel_mad'
        fmt += ', %0.3f, %0.3f, %0.3f, %0.3f'
        outbins.extend([emvel_bin_mean, emvel_bin_std, emvel_bin_med, emvel_bin_mad])

    outbins = np.ma.array(outbins).T.astype('float32')
    np.ma.set_fill_value(outbins, np.nan)
    outbins = outbins.filled(np.nan)
    if exportcsv:
        outbins_fn = os.path.join(outdir_csv, gf.feat_fn[0:8] + csv_ending)
        np.savetxt(outbins_fn, outbins, fmt=fmt, delimiter=',', header=outbins_header)
    
    
    outbins_df = pd.DataFrame(outbins, columns=outbins_header.split(','))
    return outbins_df, z_bin_edges
#     return z_bin_edges

In [2]:
#! /usr/bin/env python
"""
Compute debris thickness through sub-debris and temperature inversion methods
"""
import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from scipy import ndimage
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib
# from imview.lib import pltlib

import globaldebris_input as input

verbose=False
extra_layers=True
min_glac_area_writeout = input.min_glac_area_writeout
min_valid_area_perc = input.min_valid_area_perc
buff_dist = input.buff_dist
# bin_width = input.emvel_bin_width
#Surface to column average velocity scaling
v_col_f = input.v_col_f
csv_ending = input.output_emvel_csv_ending
emvel_filter_pixsize = input.emvel_filter_pixsize

#INPUT
glac_shp_fn_dict = {'13':input.main_directory + '/../../../HiMAT/RGI/rgi60/13_rgi60_CentralAsia/13_rgi60_CentralAsia.shp',
                    '14':input.main_directory + '/../../../HiMAT/RGI/rgi60/14_rgi60_SouthAsiaWest/14_rgi60_SouthAsiaWest.shp',
                    '15':input.main_directory + '/../../../HiMAT/RGI/rgi60/15_rgi60_SouthAsiaEast/15_rgi60_SouthAsiaEast.shp'}
glac_shp_proj_fp = input.output_fp + 'glac_shp_proj/'
if os.path.exists(glac_shp_proj_fp) == False:
    os.makedirs(glac_shp_proj_fp)


#DEM
z1_dir_sample = ('/Users/davidrounce/Documents/Dave_Rounce/HiMAT/IceThickness_Farinotti/surface_DEMs_RGI60/' + 
          'surface_DEMs_RGI60-XXXX/')
z1_fn_sample = 'surface_DEM_RGI60-XXXX.tif'
# Ice thickness
huss_dir_sample = ('/Users/davidrounce/Documents/Dave_Rounce/HiMAT/IceThickness_Farinotti/' + 
                   'composite_thickness_RGI60-all_regions/RGI60-XXXX/')
huss_fn_sample = 'RGI60-XXXX_thickness.tif'
#Surface velocity
v_dir = input.main_directory + '/../../../Satellite_Images/ITS_Live'
vx_fn = os.path.join(v_dir, 'HMA_G0120_0000_vx.tif')
vy_fn = os.path.join(v_dir, 'HMA_G0120_0000_vy.tif')

# topdir='/Users/davidrounce/Documents/Dave_Rounce/HiMAT/DEMs/'
# #Output directory
# outdir = topdir + 'Shean_2019_0213/mb_combined_20190213_nmad_bins/'
# outdir_fig = outdir + '/figures/'
outdir_csv = input.outdir_emvel_fp 
outdir_fig = input.outdir_emvel_fp  + '../figures/'

if os.path.exists(glac_shp_proj_fp) == False:
    os.makedirs(glac_shp_proj_fp)
if os.path.exists(outdir_csv) == False:
    os.makedirs(outdir_csv)
if os.path.exists(outdir_fig) == False:
    os.makedirs(outdir_fig)

In [3]:
rgiid_list = []
rgiid_fn_list = []
for i in os.listdir(input.mb_binned_fp):
    if i.endswith('mb_bins.csv'):
        rgiid_list.append(i[0:8])
        rgiid_fn_list.append(i)
        
rgiid_list = sorted(rgiid_list)
rgiid_fn_list = sorted(rgiid_fn_list)

print(len(rgiid_list))

main_glac_rgi = input.selectglaciersrgitable(rgiid_list)
main_glac_rgi['bin_fn'] = rgiid_fn_list

8053
4182 glaciers in region 13 are included in this model run: ['00062', '00093', '00130', '00135', '00137', '00140', '00147', '00175', '00181', '00183', '00203', '00210', '00277', '00358', '00382', '00386', '00391', '00394', '00400', '00401', '00403', '00439', '00440', '00441', '00465', '00561', '00585', '00594', '00604', '00606', '00611', '00628', '00643', '00693', '00713', '00750', '00751', '00757', '00761', '00763', '00777', '00788', '00809', '00830', '00834', '00838', '00880', '00884', '00885', '00891'] and more
2468 glaciers in region 14 are included in this model run: ['00005', '00018', '00032', '00036', '00043', '00057', '00063', '00072', '00088', '00101', '00104', '00111', '00131', '00142', '00145', '00146', '00159', '00163', '00164', '00187', '00213', '00219', '00222', '00225', '00235', '00243', '00251', '00271', '00287', '00309', '00323', '00326', '00336', '00346', '00347', '00352', '00353', '00363', '00366', '00367', '00370', '00372', '00380', '00398', '00403', '00432', '0

In [4]:
for nglac, glac_idx in enumerate(main_glac_rgi.index.values):
# for nglac, glac_idx in enumerate([main_glac_rgi.index.values[6940]]):
# for nglac, glac_idx in enumerate([main_glac_rgi.index.values[2896]]):
    glac_str = main_glac_rgi.loc[glac_idx,'rgino_str']
    rgiid = main_glac_rgi.loc[glac_idx,'RGIId']
    
    print(nglac, glac_idx, rgiid)
    
    out_csv_fn = os.path.join(outdir_csv, glac_str + csv_ending)
    if verbose:
        print('output_fn:', out_csv_fn)

    if not os.path.exists(out_csv_fn):
    
        region = glac_str.split('.')[0]

        # Shape layer processing
        glac_shp_init = gpd.read_file(glac_shp_fn_dict[region])
        if verbose:
            print('Shp init crs:', glac_shp_init.crs)

        glac_shp_single = glac_shp_init[glac_shp_init['RGIId'] == rgiid]
        glac_shp_single = glac_shp_single.reset_index()

        # Project shapefile
        huss_dir = huss_dir_sample.replace('XXXX',str(region.zfill(2)))
        huss_fn = huss_fn_sample.replace('XXXX',glac_str)

        proj_fn = os.path.join(huss_dir, huss_fn) # THIS PROJECTION IS KEY!
        ds = gdal.Open(proj_fn)
        prj = ds.GetProjection()
        srs = osr.SpatialReference(wkt=prj)
        aea_srs = srs

        # If projected shapefile already exists, then skip projection
        glac_shp_proj_fn = glac_shp_proj_fp + glac_str + '_crs' + str(aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp'

        if os.path.exists(glac_shp_proj_fn) == False:
            glac_shp_proj = glac_shp_single.to_crs({'init': 'epsg:' + str(aea_srs.GetAttrValue("AUTHORITY", 1))})
            glac_shp_proj.to_file(glac_shp_proj_fn)

        glac_shp_ds = ogr.Open(glac_shp_proj_fn, 0)
        glac_shp_lyr = glac_shp_ds.GetLayer()
        #This should be contained in features
        glac_shp_srs = glac_shp_lyr.GetSpatialRef()
        feat_count = glac_shp_lyr.GetFeatureCount()
        if verbose:
            print("Input glacier polygon count: %i" % feat_count)

        # Load DEM
        z1_dir = z1_dir_sample.replace('XXXX',str(region.zfill(2)))
        z1_fn = z1_fn_sample.replace('XXXX',glac_str)
        z1_ds = gdal.Open(z1_dir + z1_fn)
        z1_int_geom = geolib.ds_geom_intersection([z1_ds, z1_ds], t_srs=glac_shp_srs)

        glacfeat_list = []
        glacname_fieldname = "Name"
        #RGIId (String) = RGI50-01.00004
        glacnum_fieldname = "RGIId"
        glacnum_fmt = '%08.5f'

        for n, feat in enumerate(glac_shp_lyr):
            gf = GlacFeat(feat, glacname_fieldname, glacnum_fieldname)
            if verbose:
                print("%i of %i: %s" % (n+1, feat_count, gf.feat_fn))
            #NOTE: Input must be in projected coordinate system, ideally equal area
            #Should check this and reproject
            gf.geom_attributes(srs=aea_srs)
            glacfeat_list.append(gf)

        if verbose:
            print(gf.feat_fn)
        
        fn_dict = OrderedDict()
        #We at least want to warp the two input DEMs
        fn_dict['z1'] = os.path.join(z1_dir, z1_fn)

        if extra_layers and (gf.glac_area_km2 > min_glac_area_writeout):
            if verbose:
                print(gf.glacnum)

            # Ice thickness data
            ice_thick_fn = os.path.join(huss_dir, huss_fn)
            if os.path.exists(ice_thick_fn):
                fn_dict['ice_thick'] = ice_thick_fn

            if verbose:
                print(fn_dict['ice_thick'])

            if os.path.exists(vx_fn):
                fn_dict['vx'] = vx_fn
                fn_dict['vy'] = vy_fn

    #         if os.path.exists(ts_fullfn):
    #             fn_dict['ts'] = ts_fullfn

    #         if os.path.exists(debris_fullfn):
    #             fn_dict['debris_thick_ts'] = debris_fullfn

        #Expand extent to include buffered region around glacier polygon
        warp_extent = geolib.pad_extent(gf.glac_geom_extent, width=buff_dist)
        if verbose:
            print("Expanding extent")
            print(gf.glac_geom_extent)
            print(warp_extent)
            print(aea_srs)

        #Warp everything to common res/extent/proj
        ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res='min', \
                extent=warp_extent, t_srs=aea_srs, verbose=verbose, \
                r='cubic')

        ds_dict = dict(zip(fn_dict.keys(), ds_list))

        if verbose:
            print(ds_list)
            print(fn_dict.keys())

        #Prepare mask for all glaciers within buffered area, not just the current glacier polygon
        glac_shp_ds = ogr.Open(glac_shp_proj_fn, 0)
        glac_shp_lyr = glac_shp_ds.GetLayer()

        #Get global glacier mask
        #Want this to be True over ALL glacier surfaces, not just the current polygon
        glac_shp_lyr_mask = geolib.lyr2mask(glac_shp_lyr, ds_dict['ice_thick'])

        #Create buffer around glacier polygon
        glac_geom_buff = gf.glac_geom.Buffer(buff_dist)
        #This is False over glacier polygon surface, True elsewhere - can be applied directly
        glac_geom_buff_mask = geolib.geom2mask(glac_geom_buff, ds_dict['ice_thick'])

        # ds masks
        ds_list_masked = [iolib.ds_getma(i) for i in ds_list]
        dem1 = np.ma.masked_less_equal(ds_list_masked[0], 0)
        dems_mask = dem1.mask
        if verbose:
            print('list of datasets:', len(ds_list_masked), fn_dict.values())

        #Combine to identify ~1 km buffer around glacier polygon over static rock
        static_buffer_mask = np.ma.mask_or(~glac_shp_lyr_mask, glac_geom_buff_mask)
        static_shp_lyr_mask = np.ma.mask_or(static_buffer_mask, dems_mask)

        if 'z1' in ds_dict:
            #This is False over glacier polygon surface, True elsewhere - can be applied directly
            glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1'])
            gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']))
            #gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)

            if verbose:
                print('\n\n# z1 pixels:', gf.z1.count(), '\n')
            if gf.z1.count() == 0:
                if verbose:
                    print("No z1 pixels")
        else:
            print("Unable to load z1 ds")
        
        if nglac == 0:
            print('\n\nHACK TO BYPASS VALID AREA\n\n')
        gf.valid_area_perc = 100

        if gf.valid_area_perc < (100. * min_valid_area_perc):
            if verbose:
                print("Not enough valid pixels. %0.1f%% percent of glacier polygon area" % (gf.valid_area_perc))
        #     return None

        else:
            #Filter dz - throw out abs differences >150 m

            #Compute dz, volume change, mass balance and stats
            gf.z1_stats = malib.get_stats(gf.z1)
            z1_elev_med = gf.z1_stats[5]
            z1_elev_min, z1_elev_max = malib.calcperc(gf.z1, (0.1, 99.9))

            #Caluclate stats for aspect and slope using z2
            #Requires GDAL 2.1+
            gf.z1_aspect = np.ma.array(geolib.gdaldem_mem_ds(ds_dict['z1'], processing='aspect', returnma=True), mask=glac_geom_mask)
            gf.z1_aspect_stats = malib.get_stats(gf.z1_aspect)
            z1_aspect_med = gf.z1_aspect_stats[5]
            gf.z1_slope = np.ma.array(geolib.gdaldem_mem_ds(ds_dict['z1'], processing='slope', returnma=True), mask=glac_geom_mask)
            gf.z1_slope_stats = malib.get_stats(gf.z1_slope)
            z1_slope_med = gf.z1_slope_stats[5]

            #Can estimate ELA values computed from hypsometry and typical AAR
            #For now, assume ELA is mean
            gf.z1_ela = None
            gf.z1_ela = gf.z1_stats[3]
            #Note: in theory, the ELA should get higher with mass loss
            #In practice, using mean and same polygon, ELA gets lower as glacier surface thins


            if extra_layers and (gf.glac_area_km2 > min_glac_area_writeout):
                if 'ice_thick' in ds_dict:
                    #Load ice thickness
                    gf.H = np.ma.array(iolib.ds_getma(ds_dict['ice_thick']), mask=glac_geom_mask)
                    gf.H_mean = gf.H.mean()
                    if verbose:
                        print('mean ice thickness [m]:', gf.H_mean)

                if 'vx' in ds_dict and 'vy' in ds_dict:
                    #Load surface velocity maps
                    gf.vx = np.ma.array(iolib.ds_getma(ds_dict['vx']), mask=glac_geom_mask)
                    gf.vy = np.ma.array(iolib.ds_getma(ds_dict['vy']), mask=glac_geom_mask)
                    gf.vm = np.ma.sqrt(gf.vx**2 + gf.vy**2)
                    gf.vm_mean = gf.vm.mean()
                    if verbose:
                        print('mean velocity [m/s]:', gf.vm_mean)

                    if gf.H is not None:
                        #Compute flux
                        gf.Q = gf.H * v_col_f * np.array([gf.vx, gf.vy])
                        #Note: np.gradient returns derivatives relative to axis number, so (y, x) in this case
                        #Want x-derivative of x component
                        gf.divQ = np.gradient(gf.Q[0])[1] + np.gradient(gf.Q[1])[0]

        #                 gf.divQ = gf.H*(np.gradient(v_col_f*gf.vx)[1] + np.gradient(v_col_f*gf.vy)[0]) \
        #                         + v_col_f*gf.vx*(np.gradient(gf.H)[1]) + v_col_f*gf.vy*(np.gradient(gf.H)[0])

                        #Should smooth divQ, better handling of data gaps

                if 'ts' in ds_dict:
                    #Load surface temperature maps
                    gf.ts = np.ma.array(iolib.ds_getma(ds_dict['ts']), mask=glac_geom_mask)
                else:
                    gf.ts = None

                if 'debris_thick_ts' in ds_dict:
                    # Load debris thickness map
                    gf.debris_thick_ts = np.ma.array(iolib.ds_getma(ds_dict['debris_thick_ts']), mask=glac_geom_mask)
                    gf.meltfactor_ts = None
                else:
                    gf.debris_thick_ts = None
                    gf.meltfactor_ts = None

            if verbose:
                print('Area [km2]:', gf.glac_area / 1e6)
                print('-------------------------------')


            # Plots
    #         titles = ['Z1']
    #         z1_full2plot = gf.z1
    #         z1_full2plot.mask = dems_mask
    #         clim = malib.calcperc(z1_full2plot, (2,98))
    #         plot_array(z1_full2plot, clim, titles, 'inferno', 'Elevation (m WGS84)', fn=outdir_fig + glac_str + '_dem.png')

            #Now apply glacier mask AND mask NaN values
            glac_geom_mask = np.ma.mask_or(glac_geom_mask, dems_mask)
            # nan_mask = np.ma.masked_invalid(gf.dz)
            # glac_geom_mask = np.ma.mask_or(glac_geom_mask, nan_mask.mask)
            gf.z1 = np.ma.array(gf.z1, mask=glac_geom_mask)

            gf.res = geolib.get_res(ds_dict['z1'])

            titles = ['Z1 (masked)']
            clim = malib.calcperc(gf.z1, (2,98))
            plot_array(gf.z1, clim, titles, 'inferno', 'Elevation (m WGS84)', fn=outdir_fig + glac_str + '_dem.png')

            if verbose:
                print(gf.z1.shape)
                
#             titles = ['Vx']
#             var_full2plot = gf.vx
#             var_full2plot.mask = glac_geom_mask
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, titles, 'inferno', 'vx', fn=outdir_fig + gf.feat_fn +'_vx.png')

#             titles = ['Vy']
#             var_full2plot = gf.vy
#             var_full2plot.mask = glac_geom_mask
#             clim = malib.calcperc(var_full2plot, (2,98))
#             plot_array(var_full2plot, clim, titles, 'inferno', 'vy', fn=outdir_fig + gf.feat_fn +'_vy.png')

            gf.vtot = (gf.vx**2 + gf.vy**2)**0.5

            titles = ['Velocity (m/yr)']
            var_full2plot = gf.vtot
            var_full2plot.mask = glac_geom_mask
            clim = malib.calcperc(var_full2plot, (2,98))
            plot_array(var_full2plot, clim, titles, 'inferno', 'Velocity (m/yr)', fn=outdir_fig + glac_str +'_velocity.png')

            titles = ['Ice thickness']
            var_full2plot = gf.H
            var_full2plot.mask = glac_geom_mask
            clim = malib.calcperc(var_full2plot, (2,98))
            plot_array(var_full2plot, clim, titles, 'inferno', 'H', fn=outdir_fig + gf.feat_fn +'_ice_thickness.png')
            
#             titles = ['Flux']
#             divQ_full2plot = gf.divQ
#             divQ_full2plot.mask = glac_geom_mask
#             clim = malib.calcperc(divQ_full2plot, (2,98))
#             plot_array(divQ_full2plot, clim, titles, 'inferno', 'divQ', fn=outdir_fig + glac_str +'_divQ.png')

            # ===== "COREGISTER" SURFACE LOWERING WITH DEM USED FOR ICE THICKNESS =====
            # Load Mass Balance Data and find displacement =====
            if verbose:
                print('\nREALLY THIS SHOULD BE DONE BY COREGISTRATION OF THE TWO DEMS\n')
            mb_df_fn = main_glac_rgi.loc[glac_idx,'bin_fn']
            mb_df = pd.read_csv(input.mb_binned_fp + mb_df_fn)
            mb_df.loc[:,:] = mb_df.values.astype(np.float64)
            mb_bin0_km2 = mb_df.loc[0,' z1_bin_area_perc'] / 100 * main_glac_rgi.loc[glac_idx,'Area']
            mb_bin_size = mb_df.loc[1,'# bin_center_elev_m'] - mb_df.loc[0,'# bin_center_elev_m']
            pix_km2 = gf.res[0] * gf.res[1] / (1000)**2
            if verbose:
                print('total glacier area [km2]:', main_glac_rgi.loc[glac_idx,'Area'])
                print('initial bin area [km2]:', mb_bin0_km2)
                print('bin size [m]:', mb_bin_size)
                print('pixel size [km2]:', pix_km2)
            # Find displacement
            if len(gf.z1.compressed()) > 0:
                z1 = gf.z1.compressed()
                z1_min = z1[z1>0].min()
                z1_max = z1[z1>0].max()
                z1_km2 = 0
                elev = int(z1_min)
                while z1_km2 < mb_bin0_km2 and elev < z1_max:
                    elev += 1
                    z1_idx = np.where((z1 > 0) & (z1 < elev))
                    if len(z1_idx[0]) > 0:
                        z1_km2 = len(z1_idx[0]) * pix_km2
                #         print(elev, z1_km2)
                if verbose:
                    print(elev, z1_km2, 'vs', mb_df.loc[0,'# bin_center_elev_m'], mb_bin0_km2)
                mb_bin0_upper =  mb_df.loc[0,'# bin_center_elev_m'] + mb_bin_size / 2
                z1_offset = elev - mb_bin0_upper
                if verbose:
                    print('z1_offset:', z1_offset)
                # Update z1 with the offset
                mask_offset = np.ma.array(np.zeros(gf.z1.mask.shape) - z1_offset, mask=np.ma.getmask(gf.z1))
                gf.z1[~gf.z1.mask] = gf.z1[~gf.z1.mask] + mask_offset[~mask_offset.mask]

                # ===== EMERGENCE VELOCITY =====
                vx = np.ma.filled(gf.vx,0)
                vy = np.ma.filled(gf.vy,0)
                H = np.ma.filled(gf.H,0)
                vx[gf.z1 > gf.z1.max()] = 0
                vy[gf.z1 > gf.z1.max()] = 0
                H[gf.z1 > gf.z1.max()] = 0
                vmax = np.nanmax((vx**2 + vy**2)**0.5)

                # Emergence computation
                emvel = emergence_pixels(gf, vx, vy, H, gf.res[0], gf.res[1], 
                                         positive_is_east=True, positive_is_north=True, 
                                         constant_icethickness=False, max_velocity=vmax, vel_min=0, debug=False)
                # 3x3 filter to reduce
                if emvel_filter_pixsize > 0:
                    emvel = ndimage.filters.convolve(emvel, weights=np.full((emvel_filter_pixsize, emvel_filter_pixsize), 
                                                                            1.0/emvel_filter_pixsize**2))
                # Add to glacier feature
                gf.emvel = np.ma.masked_array(emvel, mask=np.ma.getmask(gf.z1))

                # ===== EXPORT BINNED STATISTICS =====
                #Do AED for all
                #Compute mb using scaled AED vs. polygon
                #Check for valid pixel count vs. feature area, fill if appropriate
                if gf.glac_area_km2 > min_glac_area_writeout:
                    outbins_df, z_bin_edges = hist_plot(gf, bin_width=mb_bin_size, csv_ending=csv_ending)

                    if verbose:
                        print(outbins_df.loc[0:10,['bin_center_elev_m', ' vm_med',' vm_mad', ' H_mean', ' H_std', 
                                                   ' emvel_mean', ' emvel_std',' emvel_med', ' emvel_mad']])
            else:
                print('\n' + glac_str + ' HAS NO GLACIER AREA!\n')

0 0 RGI60-13.00062
1 1 RGI60-13.00093
2 2 RGI60-13.00130
3 3 RGI60-13.00135
4 4 RGI60-13.00137
5 5 RGI60-13.00140
6 6 RGI60-13.00147
7 7 RGI60-13.00175
8 8 RGI60-13.00181
9 9 RGI60-13.00183
10 10 RGI60-13.00203
11 11 RGI60-13.00210
12 12 RGI60-13.00277
13 13 RGI60-13.00358
14 14 RGI60-13.00382
15 15 RGI60-13.00386
16 16 RGI60-13.00391
17 17 RGI60-13.00394
18 18 RGI60-13.00400
19 19 RGI60-13.00401
20 20 RGI60-13.00403
21 21 RGI60-13.00439
22 22 RGI60-13.00440
23 23 RGI60-13.00441
24 24 RGI60-13.00465
25 25 RGI60-13.00561
26 26 RGI60-13.00585
27 27 RGI60-13.00594
28 28 RGI60-13.00604
29 29 RGI60-13.00606
30 30 RGI60-13.00611
31 31 RGI60-13.00628
32 32 RGI60-13.00643
33 33 RGI60-13.00693
34 34 RGI60-13.00713
35 35 RGI60-13.00750
36 36 RGI60-13.00751
37 37 RGI60-13.00757
38 38 RGI60-13.00761
39 39 RGI60-13.00763
40 40 RGI60-13.00777
41 41 RGI60-13.00788
42 42 RGI60-13.00809
43 43 RGI60-13.00830
44 44 RGI60-13.00834
45 45 RGI60-13.00838
46 46 RGI60-13.00880
47 47 RGI60-13.00884
48 48 RGI60-

377 377 RGI60-13.06100
378 378 RGI60-13.06145
379 379 RGI60-13.06146
380 380 RGI60-13.06157
381 381 RGI60-13.06199
382 382 RGI60-13.06208
383 383 RGI60-13.06218
384 384 RGI60-13.06240
385 385 RGI60-13.06251
386 386 RGI60-13.06266
387 387 RGI60-13.06297
388 388 RGI60-13.06299
389 389 RGI60-13.06301
390 390 RGI60-13.06359
391 391 RGI60-13.06361
392 392 RGI60-13.06398
393 393 RGI60-13.06401
394 394 RGI60-13.06413
395 395 RGI60-13.06421
396 396 RGI60-13.06424
397 397 RGI60-13.06425
398 398 RGI60-13.06426
399 399 RGI60-13.06432
400 400 RGI60-13.06438
401 401 RGI60-13.06466
402 402 RGI60-13.06468
403 403 RGI60-13.06472
404 404 RGI60-13.06474
405 405 RGI60-13.06509
406 406 RGI60-13.06532
407 407 RGI60-13.06563
408 408 RGI60-13.06564
409 409 RGI60-13.06567
410 410 RGI60-13.06576
411 411 RGI60-13.06589
412 412 RGI60-13.06591
413 413 RGI60-13.06592
414 414 RGI60-13.06596
415 415 RGI60-13.06608
416 416 RGI60-13.06620
417 417 RGI60-13.06621
418 418 RGI60-13.06638
419 419 RGI60-13.06653
420 420 RGI

932 932 RGI60-13.14375
933 933 RGI60-13.14410
934 934 RGI60-13.14416
935 935 RGI60-13.14441
936 936 RGI60-13.14451
937 937 RGI60-13.14483
938 938 RGI60-13.14485
939 939 RGI60-13.14501
940 940 RGI60-13.14505
941 941 RGI60-13.14506
942 942 RGI60-13.14507
943 943 RGI60-13.14518
944 944 RGI60-13.14567
945 945 RGI60-13.14572
946 946 RGI60-13.14579
947 947 RGI60-13.14587
948 948 RGI60-13.14589
949 949 RGI60-13.14593
950 950 RGI60-13.14612
951 951 RGI60-13.14624
952 952 RGI60-13.14626
953 953 RGI60-13.14628
954 954 RGI60-13.14643
955 955 RGI60-13.14646
956 956 RGI60-13.14662
957 957 RGI60-13.14675
958 958 RGI60-13.14678
959 959 RGI60-13.14690
960 960 RGI60-13.14704
961 961 RGI60-13.14711
962 962 RGI60-13.14715
963 963 RGI60-13.14717
964 964 RGI60-13.14738
965 965 RGI60-13.14739
966 966 RGI60-13.14759
967 967 RGI60-13.14771
968 968 RGI60-13.14789
969 969 RGI60-13.14797
970 970 RGI60-13.14799
971 971 RGI60-13.14829
972 972 RGI60-13.14839
973 973 RGI60-13.14852
974 974 RGI60-13.14859
975 975 RGI

1345 1345 RGI60-13.19806
1346 1346 RGI60-13.19807
1347 1347 RGI60-13.19824
1348 1348 RGI60-13.19847
1349 1349 RGI60-13.19851
1350 1350 RGI60-13.19863
1351 1351 RGI60-13.19869
1352 1352 RGI60-13.19874
1353 1353 RGI60-13.19875
1354 1354 RGI60-13.19878
1355 1355 RGI60-13.19891
1356 1356 RGI60-13.19897
1357 1357 RGI60-13.19920
1358 1358 RGI60-13.19930
1359 1359 RGI60-13.19942
1360 1360 RGI60-13.19943
1361 1361 RGI60-13.19944
1362 1362 RGI60-13.19981
1363 1363 RGI60-13.20005
1364 1364 RGI60-13.20012
1365 1365 RGI60-13.20035
1366 1366 RGI60-13.20038
1367 1367 RGI60-13.20043
1368 1368 RGI60-13.20074
1369 1369 RGI60-13.20111
1370 1370 RGI60-13.20112
1371 1371 RGI60-13.20114
1372 1372 RGI60-13.20139
1373 1373 RGI60-13.20143
1374 1374 RGI60-13.20147
1375 1375 RGI60-13.20164
1376 1376 RGI60-13.20170
1377 1377 RGI60-13.20175
1378 1378 RGI60-13.20177
1379 1379 RGI60-13.20179
1380 1380 RGI60-13.20180
1381 1381 RGI60-13.20210
1382 1382 RGI60-13.20217
1383 1383 RGI60-13.20218
1384 1384 RGI60-13.20222


1731 1731 RGI60-13.26642
1732 1732 RGI60-13.26661
1733 1733 RGI60-13.26666
1734 1734 RGI60-13.26669
1735 1735 RGI60-13.26671
1736 1736 RGI60-13.26672
1737 1737 RGI60-13.26713
1738 1738 RGI60-13.26892
1739 1739 RGI60-13.26898
1740 1740 RGI60-13.26904
1741 1741 RGI60-13.26906
1742 1742 RGI60-13.26909
1743 1743 RGI60-13.26910
1744 1744 RGI60-13.26911
1745 1745 RGI60-13.26922
1746 1746 RGI60-13.26923
1747 1747 RGI60-13.26926
1748 1748 RGI60-13.26927
1749 1749 RGI60-13.26928
1750 1750 RGI60-13.26929
1751 1751 RGI60-13.26931
1752 1752 RGI60-13.26932
1753 1753 RGI60-13.26941
1754 1754 RGI60-13.26949
1755 1755 RGI60-13.26953
1756 1756 RGI60-13.26954
1757 1757 RGI60-13.26961
1758 1758 RGI60-13.26966
1759 1759 RGI60-13.26967
1760 1760 RGI60-13.26968
1761 1761 RGI60-13.26969
1762 1762 RGI60-13.26972
1763 1763 RGI60-13.26974
1764 1764 RGI60-13.26985
1765 1765 RGI60-13.27024
1766 1766 RGI60-13.27035
1767 1767 RGI60-13.27038
1768 1768 RGI60-13.27113
1769 1769 RGI60-13.27118
1770 1770 RGI60-13.27183


...

6631 6631 RGI60-14.27652
6632 6632 RGI60-14.27657
6633 6633 RGI60-14.27664
6634 6634 RGI60-14.27671
6635 6635 RGI60-14.27678
6636 6636 RGI60-14.27713
6637 6637 RGI60-14.27786
6638 6638 RGI60-14.27791
6639 6639 RGI60-14.27796
6640 6640 RGI60-14.27801
6641 6641 RGI60-14.27863
6642 6642 RGI60-14.27942
6643 6643 RGI60-14.27943
6644 6644 RGI60-14.27946
6645 6645 RGI60-14.27951
6646 6646 RGI60-14.27954
6647 6647 RGI60-14.27955
6648 6648 RGI60-14.27959
6649 6649 RGI60-14.27975
6650 6650 RGI60-15.00024
6651 6651 RGI60-15.00026
6652 6652 RGI60-15.00055
6653 6653 RGI60-15.00057
6654 6654 RGI60-15.00107
6655 6655 RGI60-15.00186
6656 6656 RGI60-15.00194
6657 6657 RGI60-15.00232
6658 6658 RGI60-15.00233
6659 6659 RGI60-15.00234
6660 6660 RGI60-15.00288
6661 6661 RGI60-15.00355
6662 6662 RGI60-15.00356
6663 6663 RGI60-15.00358
6664 6664 RGI60-15.00368
6665 6665 RGI60-15.00372
6666 6666 RGI60-15.00379
6667 6667 RGI60-15.00399
6668 6668 RGI60-15.00406
6669 6669 RGI60-15.00410
6670 6670 RGI60-15.00423


6959 6959 RGI60-15.03618
6960 6960 RGI60-15.03619
6961 6961 RGI60-15.03673
6962 6962 RGI60-15.03684
6963 6963 RGI60-15.03692
6964 6964 RGI60-15.03697
6965 6965 RGI60-15.03704
6966 6966 RGI60-15.03706
6967 6967 RGI60-15.03728
6968 6968 RGI60-15.03731
6969 6969 RGI60-15.03733
6970 6970 RGI60-15.03734
6971 6971 RGI60-15.03735
6972 6972 RGI60-15.03741
6973 6973 RGI60-15.03742
6974 6974 RGI60-15.03743
6975 6975 RGI60-15.03744
6976 6976 RGI60-15.03765
6977 6977 RGI60-15.03782
6978 6978 RGI60-15.03784
6979 6979 RGI60-15.03830
6980 6980 RGI60-15.03833
6981 6981 RGI60-15.03837
6982 6982 RGI60-15.03841
6983 6983 RGI60-15.03850
6984 6984 RGI60-15.03923
6985 6985 RGI60-15.03948
6986 6986 RGI60-15.03954
6987 6987 RGI60-15.03973
6988 6988 RGI60-15.03975
6989 6989 RGI60-15.03976
6990 6990 RGI60-15.03981
6991 6991 RGI60-15.03986
6992 6992 RGI60-15.04005
6993 6993 RGI60-15.04018
6994 6994 RGI60-15.04023
6995 6995 RGI60-15.04024
6996 6996 RGI60-15.04046
6997 6997 RGI60-15.04060
6998 6998 RGI60-15.04061


7287 7287 RGI60-15.06728
7288 7288 RGI60-15.06730
7289 7289 RGI60-15.06741
7290 7290 RGI60-15.06754
7291 7291 RGI60-15.06763
7292 7292 RGI60-15.06774
7293 7293 RGI60-15.06776
7294 7294 RGI60-15.06777
7295 7295 RGI60-15.06789
7296 7296 RGI60-15.06807
7297 7297 RGI60-15.06816
7298 7298 RGI60-15.06817
7299 7299 RGI60-15.06826
7300 7300 RGI60-15.06828
7301 7301 RGI60-15.06845
7302 7302 RGI60-15.06854
7303 7303 RGI60-15.06861
7304 7304 RGI60-15.06881
7305 7305 RGI60-15.06884
7306 7306 RGI60-15.06889
7307 7307 RGI60-15.06896
7308 7308 RGI60-15.06941
7309 7309 RGI60-15.06961
7310 7310 RGI60-15.06977
7311 7311 RGI60-15.06978
7312 7312 RGI60-15.06981
7313 7313 RGI60-15.06987
7314 7314 RGI60-15.06998
7315 7315 RGI60-15.07024
7316 7316 RGI60-15.07025
7317 7317 RGI60-15.07034
7318 7318 RGI60-15.07036
7319 7319 RGI60-15.07042
7320 7320 RGI60-15.07044
7321 7321 RGI60-15.07071
7322 7322 RGI60-15.07072
7323 7323 RGI60-15.07073
7324 7324 RGI60-15.07075
7325 7325 RGI60-15.07084
7326 7326 RGI60-15.07086


7615 7615 RGI60-15.10054
7616 7616 RGI60-15.10055
7617 7617 RGI60-15.10056
7618 7618 RGI60-15.10062
7619 7619 RGI60-15.10070
7620 7620 RGI60-15.10076
7621 7621 RGI60-15.10088
7622 7622 RGI60-15.10105
7623 7623 RGI60-15.10115
7624 7624 RGI60-15.10126
7625 7625 RGI60-15.10130
7626 7626 RGI60-15.10135
7627 7627 RGI60-15.10146
7628 7628 RGI60-15.10147
7629 7629 RGI60-15.10154
7630 7630 RGI60-15.10161
7631 7631 RGI60-15.10180
7632 7632 RGI60-15.10181
7633 7633 RGI60-15.10188
7634 7634 RGI60-15.10190
7635 7635 RGI60-15.10193
7636 7636 RGI60-15.10196
7637 7637 RGI60-15.10205
7638 7638 RGI60-15.10209
7639 7639 RGI60-15.10211
7640 7640 RGI60-15.10222
7641 7641 RGI60-15.10232
7642 7642 RGI60-15.10240
7643 7643 RGI60-15.10243
7644 7644 RGI60-15.10247
7645 7645 RGI60-15.10248
7646 7646 RGI60-15.10254
7647 7647 RGI60-15.10255
7648 7648 RGI60-15.10261
7649 7649 RGI60-15.10269
7650 7650 RGI60-15.10270
7651 7651 RGI60-15.10271
7652 7652 RGI60-15.10273
7653 7653 RGI60-15.10275
7654 7654 RGI60-15.10276


7943 7943 RGI60-15.12414
7944 7944 RGI60-15.12424
7945 7945 RGI60-15.12429
7946 7946 RGI60-15.12432
7947 7947 RGI60-15.12434
7948 7948 RGI60-15.12435
7949 7949 RGI60-15.12438
7950 7950 RGI60-15.12439
7951 7951 RGI60-15.12448
7952 7952 RGI60-15.12454
7953 7953 RGI60-15.12457
7954 7954 RGI60-15.12462
7955 7955 RGI60-15.12464
7956 7956 RGI60-15.12470
7957 7957 RGI60-15.12479
7958 7958 RGI60-15.12492
7959 7959 RGI60-15.12493
7960 7960 RGI60-15.12498
7961 7961 RGI60-15.12500
7962 7962 RGI60-15.12502
7963 7963 RGI60-15.12504
7964 7964 RGI60-15.12506
7965 7965 RGI60-15.12512
7966 7966 RGI60-15.12517
7967 7967 RGI60-15.12520
7968 7968 RGI60-15.12526
7969 7969 RGI60-15.12535
7970 7970 RGI60-15.12536
7971 7971 RGI60-15.12540
7972 7972 RGI60-15.12543
7973 7973 RGI60-15.12544
7974 7974 RGI60-15.12547
7975 7975 RGI60-15.12549
7976 7976 RGI60-15.12550
7977 7977 RGI60-15.12552
7978 7978 RGI60-15.12553
7979 7979 RGI60-15.12566
7980 7980 RGI60-15.12579
7981 7981 RGI60-15.12582
7982 7982 RGI60-15.12585
